# Исследование надёжности заёмщиков

**Заказчик исследования** — кредитный отдел банка


**Цель исследования** — понять, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок,  в частности:
  1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
  2. Есть ли зависимость между семейным положением и возвратом кредита в срок? 
  3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
  4. Как разные цели кредита влияют на его возврат в срок?

Результаты исследования будут учтены при построении модели кредитного скоринга, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**План исследования:**
 1. Обзор данных.
 2. Предобработка данных (поиск и замена пропущенных значений, аномалий, дубликатов).
 3. Проверка гипотез.

## 1. Обзор данных

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [2]:
try:
    borrowers = pd.read_csv('/Users/galina/yandex-praktikum-projects/data.csv')
except:
    borrowers = pd.read_csv('/datasets/data.csv')

In [3]:
#смотрю первые 5 строк:
borrowers.head(5)  

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
#смотрю последние 5 строк:
borrowers.tail(5) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [5]:
#смотрю общую инфо по таблице:
borrowers.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Выводы:
1. Количество значений в столбцах различается (например, в столбце `children` 21 525 строк, а в столбце `days_employed` и столбце `total_income` - 19 351). Значит, в данных есть пропущенные значения.
2. Есть артефакты, например, отрицательное количество дней трудового стажа в столбце `days_employed`, стаж у пенсионеров в столбце `days_employed` более 1000 лет. Значит, и в других столбцах могут быть артефакты
3. Тип данных в части столбцов не соответствует данным, например, в столбце `total_income` должен быть тип данных `int` или `float`.
4. Есть дубликаты, например, в столбце `education` одни и те же значения записаны либо строчными буквами, либо капслоком, либо с заглавной буквы.
5. Сама таблица большая, для удобства работы ее можно разделить на: 1) два "словаря" со столбцами `education`, `education_id` и `family_status`, `family_status_id`, 2) с основными данными. 

Данных достаточно для проверки гипотез, но встречаются пропуски в данных, дубли и ошибки в данных. По возможности надо запросить недостающие данные из CRM / CDP (такое бывает при "зоопарке" систем - когда одна получает лиды, другая - клиентов, и при передаче данных из одной в другую появляются ошибки).

## 2. Предобработка данных

### 2.1. Пропуски значений

In [6]:
borrowers.isna().sum()  #смотрю, сколько и в каких столбцах пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбце `total_income` и столбце `days_employed` - по 2 174 пропуска значений.<br>

In [7]:
borrowers[(borrowers['days_employed'].isnull() == True) & (borrowers['total_income'].isnull() == True)].info()  #добавила этот код, чтобы проверить строки по обоим столбцам, чтобы понять, в одних и тех же ли строках пропуски

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


В строках, где отсутствуют данные в столбце `days_employed`, отсутствуют данные и по `total_income`. Возможно, из-за зависимости значений в одном столбце от значений в другом столбце сбой в одном столбце повлек за собой сбой в другом, поэтому мы получили пустые строки в обоих столбцах. <br>

Смотрю максимальное и минимальное значение в столбце `total_income`, чтобы понять "амплитуду" данных:

In [8]:
borrowers['total_income'].max()

2265604.028722744

In [9]:
borrowers['total_income'].min() 

20667.26379327158

Возможная причина пропуска данных в столбце `total_income` - ошибка при получении или обработке данных. Также причиной пропуска может быть невнесение данных в CRM / CDP, но с гораздо меньшей вероятностью, так как данные по доходу нужны для оценки заемщика и расчета конечной ставку по кредиту. Сокрытие данных о доходе со стороны заемщика, как причину пропуска данных, можно отмести, так как без этих данных заявку на кредит подать можно только в МФО, не в банк.

2 174 пропуска - это 10% от общего кол-ва строк, удаление 10% данных означает искажение результата на выходе. <br>Следовательно, пропуски надо заполнить. 

Для начала проверю, из разных ли это `income_type` заемщики:

In [10]:
borrowers[(borrowers['total_income'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Заемщики, у которых нет данных по `total_income`, имеют разные `income_type`. Поэтому надо искать медианный доход по каждой группе `income_type`. Медиана не даст большого искажения, в отличие от среднего арифметического.

In [11]:
#вычисляю медианный доход по типу занятости 'сотрудник':
total_income_employee_avg = \
borrowers[(borrowers['income_type']=='сотрудник')]['total_income'].median()  

#вычисляю медианный доход по типу занятости 'компаньон':
total_income_companion_avg = \
borrowers[(borrowers['income_type']=='компаньон')]['total_income'].median()  

#вычисляю медианный доход по типу занятости 'пенсионер':
total_income_retiree_avg = \
borrowers[(borrowers['income_type']=='пенсионер')]['total_income'].median()  

#вычисляю медианный доход по типу занятости 'госслужащий':
total_income_state_employee_avg = \
borrowers[(borrowers['income_type']=='госслужащий')]['total_income'].median()  

#вычисляю медианный доход по типу занятости 'предприниматель':
total_income_enterpreneur_avg = \
borrowers[(borrowers['income_type']=='предприниматель')]['total_income'].median()  

#вывожу посмотреть медианы:
display(total_income_employee_avg,
        total_income_companion_avg,
        total_income_retiree_avg,
        total_income_state_employee_avg,
        total_income_enterpreneur_avg)  

142594.39684740017

172357.95096577113

118514.48641164352

150447.9352830068

499163.1449470857

Кстати, предпринимателей в таблице всего 2.
Я бы удалила обоих, так как вычислять на основе 1 ячейки медианный доход и "присваивать" его второй ячейке, статистически странно. Кроме того, удаление 2 заемщиков (меньше сотой процента об общего кол-ва заемщиков в таблице) ничего не исказит.   
Но ради тренировки оставила.

Теперь заменяю медианным доходом пропуски доходов в строках `сотрудник`, `компаньон`, `пенсионер`, `госслужащий`, `предприниматель`:

In [12]:
borrowers.loc[borrowers['income_type']=='сотрудник','total_income'] = \
borrowers[(borrowers['income_type']=='сотрудник')]['total_income'].fillna(value=total_income_employee_avg)

borrowers.loc[borrowers['income_type']=='компаньон','total_income'] = \
borrowers[(borrowers['income_type']=='компаньон')]['total_income'].fillna(value=total_income_companion_avg)

borrowers.loc[borrowers['income_type']=='пенсионер','total_income'] = \
borrowers[(borrowers['income_type']=='пенсионер')]['total_income'].fillna(value=total_income_retiree_avg)

borrowers.loc[borrowers['income_type']=='госслужащий','total_income'] = \
borrowers[(borrowers['income_type']=='госслужащий')]['total_income'].fillna(value=total_income_state_employee_avg)

borrowers.loc[borrowers['income_type']=='предприниматель','total_income'] = \
borrowers[(borrowers['income_type']=='предприниматель')]['total_income'].fillna(value=total_income_enterpreneur_avg)

#проверяю, заменились ли пропуски в столбце 'total_income' на медианы
borrowers.isna().sum()  

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [13]:
borrowers['total_income'] = \
borrowers['total_income'].fillna(borrowers.groupby('income_type')['total_income'].transform('median'))

Вернусь к столбцу `days_employed`:

In [14]:
#смотрю максимальное значение в столбце, чтобы понять разницу в данных
borrowers['days_employed'].max() 

401755.40047533

Хм, более 1000 лет стажа...

In [15]:
#смотрю минимальное значение в столбце, чтобы понять разницу в данных
borrowers['days_employed'].min() 

-18388.949900568383

Отрицательный стаж!

Возможная причина пропуска данных, завышенных данных, отрицательных значений в столбце `days_employed` - ошибка при получении или обработке данных. Отрицательные значения могли возникнуть из-за ошибки при внесении данных (думали, что ставят тире, а ставили минус).

2 174 пропуска - это 10% от общего кол-ва строк, удаление 10% данных означает искажение результата на выходе. Поэтому:
<br>- заменю отрицательные значения на положительные, 
<br>- заполню пропуски медианными значениями по столбцу.

Сначала заменяю отрицательные значения в столбце `days_employed` на положительные:

In [16]:
#избавляюсь от минуса за счет модуля:
borrowers['days_employed'] = borrowers['days_employed'].abs() 

#проверяю, есть ли отрицательные значения в столбце:
borrowers[borrowers['days_employed']<=0] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Пропуски в стаже должны совпать с пропусками в доходе по `income_type`, так как ранее я выяснила, что пропуски по столбцу `total_income` и  `days_employed` совпадают, но проверю:

In [17]:
borrowers[(borrowers['days_employed'].isnull()==True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [18]:
#вычисляю медианный стаж для групп:
days_employed_employee_avg = \
borrowers[(borrowers['income_type']=='сотрудник')]['days_employed'].median()  

days_employed_companion_avg = \
borrowers[(borrowers['income_type']=='компаньон')]['days_employed'].median()  

days_employed_retiree_avg = \
borrowers[(borrowers['income_type']=='пенсионер')]['days_employed'].median()  

days_employed_state_employee_avg = \
borrowers[(borrowers['income_type']=='госслужащий')]['days_employed'].median()  

days_employed_enterpreneur_avg = \
borrowers[(borrowers['income_type']=='предприниматель')]['days_employed'].median()  

display(days_employed_employee_avg,
        days_employed_companion_avg,
        days_employed_retiree_avg,
        days_employed_state_employee_avg,
        days_employed_enterpreneur_avg)  

1574.2028211070854

1547.3822226779334

365213.3062657312

2689.3683533043886

520.8480834953765

У пенсионеров нереалистично огромный стаж. Разберусь с этим ниже. 
Пока заменю медианным стажем пропуски

In [19]:
borrowers.loc[borrowers['income_type']=='сотрудник','days_employed'] = \
borrowers[(borrowers['income_type'] == 'сотрудник')]['days_employed'].fillna(value=days_employed_employee_avg)

borrowers.loc[borrowers['income_type']=='компаньон','days_employed'] = \
borrowers[(borrowers['income_type'] == 'компаньон')]['days_employed'].fillna(value=days_employed_companion_avg)

borrowers.loc[borrowers['income_type']=='пенсионер','days_employed'] = \
borrowers[(borrowers['income_type'] == 'пенсионер')]['days_employed'].fillna(value=days_employed_retiree_avg)

borrowers.loc[borrowers['income_type']=='госслужащий','days_employed'] = \
borrowers[(borrowers['income_type'] == 'госслужащий')]['days_employed'].fillna(value=days_employed_state_employee_avg)

borrowers.loc[borrowers['income_type']=='предприниматель','days_employed'] = \
borrowers[(borrowers['income_type'] == 'предприниматель')]['days_employed'].fillna(value=days_employed_enterpreneur_avg)

borrowers.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Вернусь к завышенным данным в столбце `days_employed`. Несмотря на то, что исследование зависимости между длиной стажа и возвратом кредита в срок не входит в задачи, хочу посмотреть на эти артефакты, их кол-во и долю от общего кол-ва заемщиков, и заменить на реалистичные.

Так как стаж явно не может быть дольше 1100 лет, поэтому поищу долю тех, у которых стаж выше 50 лет (12 350 дней при 247 рабочих днях в году в среднем):

In [20]:
count_experience50 = 0

for i in borrowers['days_employed']:
    if i > 12350:
        count_experience50 += 1

total = borrowers['days_employed'].count()    
count_experience50 / total

0.18276422764227643

То есть заемщиков, у которых стаж выше реалистичных 50 лет - 18% от общего кол-ва заемщиков.
Посмотрю, в каких группах `income_type` эти нарушения:

In [21]:
borrowers[(borrowers['days_employed']>12350)]['income_type'].value_counts()

пенсионер      3856
сотрудник        49
компаньон        14
госслужащий      13
безработный       2
Name: income_type, dtype: int64

Так как:
- у нас есть данные по возрасту заемщиков,
- в среднем люди идут работать в 18,
- пенсионный наступает в 65 лет,
<br> то можно вычислить примерный близкий к реалистичному стаж для каждого заемщика, у которого в текущей версии он превышает 12350 дней. 

<br>(Но лучше запросить дополнительные данные).

In [22]:
borrowers.loc[(borrowers['days_employed']>12350) &
              (borrowers['income_type']=='пенсионер'),
              ['days_employed']] = 11609

borrowers.loc[(borrowers['days_employed']>12350) &
              (borrowers['income_type']!='пенсионер'),
              ['days_employed']] = (borrowers['dob_years']-18)*247

### 2.2. Замена значений

In [23]:
borrowers['children'].value_counts()  #смотрю распределение по кол-ву детей

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Явно есть ошибки -  у 47 заемщиков по -1 ребенку и у 76 заемщиков по 20 детей.
<br>В случае с -1 ребенком заменю -1 на 1 (возможно, причина ошибки - человеческий фактор, ошиблись при вводе значения, просто добавив тире, которое было считано как минус).
<br>В случае с 20 детьми заменю 20 на медианное значение по другим строкам (возможно, причина ошибки - человеческий фактор, либо хотели ввести 0, либо 2).

In [24]:
#заменяю -1 ребенка на 1 ребенка:
borrowers['children'] = borrowers['children'].replace(-1, 1)  

#вычисляю медианное кол-во детей по строкам, где не 20 детей 
children_median = borrowers.loc[borrowers.loc[:, 'children']!=20]['children'].median()  

#заменяю 20 детей на медианное кол-во детей
borrowers['children'] = borrowers['children'].replace(20, children_median)  

#проверяю кол-во детей
borrowers['children'].value_counts()  

0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Что в других столбцах? 

In [25]:
borrowers['dob_years'].value_counts() 

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

У 101 заемщика возраст 0. Если бы в задачу входило исследование зависимости между возрастом и возвратом кредита в срок, то эту ошибку стоило бы поправить, запросив данные заново и/или исправив их для других аналитиков. Но так как такой задачи нет, пойдем дальше и посмотрим, как там с другими столбцами.

In [26]:
#смотрю уникальные статусы образования и кол-во:
borrowers['education'].value_counts()  

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Есть неявные дубликаты:
* *ВЫСШЕЕ, Высшее и высшее*
* *НЕОКОНЧЕННОЕ ВЫСШЕЕ, Неоконченное высшее и неоконченное высшее*
* *НАЧАЛЬНОЕ, Начальное и начальное*
* *СРЕДНЕЕ, Среднее и среднее*
* *УЧЕНАЯ СТЕПЕНЬ, Ученая степень и ученая степень*.<br>

От них избавляюсь в п.2.4. Удаление дубликатов.

In [27]:
borrowers['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Тут дубликатов нет (но можно заменить "Не женат" на "не женат")

In [28]:
borrowers['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

А тут какая-то ошибка - есть один заемщик, пол которого не F и не M. Так как ФИО заемщиков не указаны и задачи проверить зависимость между полом и возвратом кредита нет, то можно пойти дальше или удалить. Удаляю:

In [29]:
#удалила строку с непонятным гендером
borrowers = borrowers.drop(borrowers[borrowers['gender'] == 'XNA'].index)  

#проверяю, удалила ли
borrowers['gender'].value_counts()  

F    14236
M     7288
Name: gender, dtype: int64

In [30]:
borrowers['income_type'].value_counts()

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Тут дубликатов нет.

In [31]:
borrowers['debt'].value_counts()

0    19783
1     1741
Name: debt, dtype: int64

У 8,8% была задолженность.

In [32]:
borrowers['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      623
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Тут цели кредита можно типировать. Этим займусь в п.3. Проверка зависимостей.

### 2.3. Замена типов данных 

В столбце `total_income` заменяю вещественный тип данных на целочисленный, так как копейки не влияют на результат.

In [33]:
#заменяю вещественный тип данных на целочисленный
borrowers['total_income'] = borrowers['total_income'].astype('int') 

#проверяю первые 10 значений столбца
borrowers['total_income'].head(10)  

0    253875
1    112080
2    145885
3    267628
4    158616
5    255763
6    240525
7    135823
8     95856
9    144425
Name: total_income, dtype: int64

В столбце `days_employed` заменяю вещественный тип данных на целочисленный, так как дробная часть незначительно влияет на результат, но усложняет визуальный анализ.

In [34]:
#заменяю вещественный тип данных на целочисленный
borrowers['days_employed'] = borrowers['days_employed'].astype('int') 

#проверяю первые 10 значений столбца
borrowers['days_employed'].head(10)  

0     8437
1     4024
2     5623
3     4124
4    11609
5      926
6     2879
7      152
8     6929
9     2188
Name: days_employed, dtype: int64

### 2.4. Удаление дубликатов

Избавляюсь от неявных дубликатов в столбце `education`, выявленных выше:

In [35]:
#перевожу все данные в столбце к одному регистру
borrowers['education'] = borrowers['education'].str.lower()  

#вывожу данные по столбцу, чтобы посмотреть еше раз уникальные значения и проверить дубликаты
borrowers['education'].value_counts() 

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [36]:
#проверяю, есть ли явные дубликаты и сколько
borrowers.duplicated().sum()  

71

Явные дубликаты могли возникнуть из-за объединения двух и более таблиц или из-за технических проблем, неявные - из-за человеческого фактора. 
<br>Для избавления от неявных с помощью value_counts() посмотрела уникальные статусы образования и кол-во. Поскольку дубликаты отличались друг от друга только регистрами, то применила str.lower(). 
<br>Для избавления от явных дубликатов я применяю обычный drop_duplicates() с формированием новых индексов:

In [37]:
#удаляю явные дубликаты (с формированием новых индексов)
borrowers = borrowers.drop_duplicates().reset_index(drop=True) 

#считаю явные дубликаты, чтобы понять, избавилась ли от них
borrowers.duplicated().sum()  

0

### 2.5. Категоризация

Создам 2 отдельных «словаря», где:
<br>1) каждому уникальному значению из education соответствует уникальное значение education_id
<br>2) каждому уникальному значению из family_status соответствует уникальное значение family_status_id
<br>чтобы оставить в таблице education_id и family_status_id. Это упростит работу с таблицей.

In [38]:
#составляю "словарь", который соотносит education и education_id без дубликатов
education_dict = borrowers[['education','education_id']].drop_duplicates().reset_index(drop=True) 

#вывожу, чтобы посмотреть результат
education_dict.head()  

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [39]:
family_status_dict = borrowers[['family_status','family_status_id']].drop_duplicates().reset_index(drop=True)
family_status_dict.head()

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [40]:
#составляю новую таблицу без 'family_status' и без 'education' 
borrowers_log = borrowers[['children',
                           'days_employed',
                           'dob_years',
                           'education_id',
                           'family_status_id',
                           'gender',
                           'income_type',
                           'debt',
                           'total_income',
                           'purpose']]  
borrowers_log.head(10) 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,11609,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


Категоризирую заемщиков по уровню ежемесячного дохода.

In [41]:
#создаю функцию, определяющую категорию заемщика в зависимости от ежемесячного дохода

def total_income_category(category):  
    if 0 <= category <= 30000:
        return 'E'
    
    if 30001 <= category <= 50000:
        return 'D'
    
    if 50001 <= category <= 200000:
        return 'C'
    
    if 200001 <= category <= 1000000:
        return 'B'
    
    if category >= 1000001:
        return 'A'

borrowers_log['total_income_category'] = borrowers_log['total_income'].apply(total_income_category)

borrowers_log

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,11609,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B
21449,0,11609,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21450,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C
21451,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


Так как ранее я увидела, что многие цели кредита названы очень похоже, можно разбить их на категории.

In [42]:
#создаю функцию, которая на основании purpose формирует purpose_category

def purpose_category(category):  
        if 'свадьб' in category:
            return 'проведение свадьбы'
    
        if 'автомобил' in category:
            return 'операции с автомобилем'
   
        if 'недвижимост' in category or 'жил' in category:
            return 'операции с недвижимостью'
        
        if 'образован' in category:
            return 'получение образования'
        
borrowers_log['purpose_category'] = borrowers_log['purpose'].apply(purpose_category)  #формирую столбец purpose_category
borrowers_log

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,11609,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21449,0,11609,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21450,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21451,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


### Выводы по п.2. Предобработка данных
<br>Без пропусков, без дубликатов, с нужным типом данных результат исследования должен стать точнее.
<br>С разбивкой на основную таблицу и "словари" работать удобнее и быстрее.
<br>Теперь можно перейти к проверке зависимостей.

 ## 3. Проверка зависимостей

### 3.1. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Теперь можно сделать сводную таблицу:

In [43]:
debt_total_income = borrowers_log.pivot_table(index='total_income_category',
                                              values='debt',
                                              aggfunc=['count', 'sum','mean']) 
debt_total_income

,count,sum,mean
,debt,debt,debt
total_income_category,,,
A,25,2,0.080000
B,5041,356,0.070621
C,16015,1360,0.084920
D,350,21,0.060000
E,22,2,0.090909


Зависимости между уровнем дохода и уходом в просрочку нет, так как в просрочка 6-9% вне зависимости от категории.

### 3.2. Как разные цели кредита влияют на его возврат в срок?

In [44]:
debt_purpose_category = borrowers_log.pivot_table(index='purpose_category',
                                              values='debt',
                                              aggfunc=['count', 'sum','mean']) 
debt_purpose_category

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4306,403,0.093590
операции с недвижимостью,10810,782,0.072340
получение образования,4013,370,0.092200
проведение свадьбы,2324,186,0.080034


Наименьшая просрочка в категории "операции с недвижимостью", наибольшая - "получение образования" и "операции с автомобилем"

### 3.3. Есть ли зависимость между количеством детей и возвратом кредита в срок?

Так как дети уже сами себя категоризировали по кол-ву, то можно сразу составлять сводную таблицу:

Делаю то же с использованием метода `.pivot_table`:

In [45]:
debt_children = borrowers_log.pivot_table(index='children',
                                              values='debt',
                                              aggfunc=['count', 'sum','mean']) 
debt_children

,count,sum,mean
,debt,debt,debt
children,,,
0,14166,1071,0.075604
1,4855,445,0.091658
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


Наименьшая просрочка у заемщиков c 5 детьми, однако их всего 9 шт, то есть результат может быть непоказательный.<br> Явной зависимости между кол-вом детей и возвратом кредита в срок не вижу, так как у заемщиков с 1-3 детьми примерно одинаковый уровень задолженности. Только у замщиков с 3 детьми немного меньше просрочка 

### 3.4. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [46]:
debt_family_status_id = borrowers_log.pivot_table(index='family_status_id',
                                              values='debt',
                                              aggfunc=['count', 'sum','mean']) 
debt_family_status_id

,count,sum,mean
,debt,debt,debt
family_status_id,,,
0,12339,931,0.075452
1,4150,388,0.093494
2,959,63,0.065693
3,1195,85,0.071130
4,2810,274,0.097509


In [47]:
borrowers[['family_status','family_status_id']].value_counts()

family_status          family_status_id
женат / замужем        0                   12339
гражданский брак       1                    4150
Не женат / не замужем  4                    2810
в разводе              3                    1195
вдовец / вдова         2                     959
dtype: int64

Максимальная просрочка у неженатых / незамужних заемщиков и тех, кто в гражданском браке. Минимальная - у вдовцов и разведенных. То есть при скоринге стоит обратить внимание на семейный статус и повышать ставку тз-за риска просрочки неженатым / незамужним заемщиков и тем, кто в гражданском браке

## Итоги исследования:

Общий уровень просроченной задолженности - 8,8%. При этом наименее ответственно погашают кредиты и, соответсвенно, уходят в просрочку:
1) заемщики с ежемесячным доходом менее 30 тыс. рублей и с доходом от 50 до 200 тыс. рублей,<br>
2) взявшие кредит на получение образования и операции с автомобилем,<br>
3) заемщики с 4 детьми,<br>
4) неженатые / незамужние заемщики.<br>

При этом явной зависимости между возвратом кредита в срок и доходом / семейным статусом / детьми / целью кредита не выявила.

Однако в целом 8,8% - это высокий уровень просроченной задолженности, следовательно, повод проанализировать: <br>
1) кредитный портфель банка в целом и в частности по бОльшей выборке заемщиков-физлиц, <br>
2) на основании результатов этого исследования, возможно, изменить кредитную политику банка в отношении физлиц (либо требования жестче, либо фонды больше).